In [1]:
import numpy as np
import random
import tqdm

from utils import separate_dots, shuffle_dataset, generate_dots

### parameters of the lab

In [2]:
epochs = 20
n_samples = 20000000
epsilon = 1e-4
radius_start, radius_end = 1, 7
center_start_x, center_end_x = -10, 10
center_start_y, center_end_y = -10, 10

###  generate radius and center
(x_min,y_min), (x_max, y_max) - it's coordinates of the bounding box where points will be generated inside of

In [3]:
radius = np.random.rand()*(radius_end - radius_start) + radius_start
center_x = np.random.rand()*(center_end_x - center_start_x) + center_start_x
center_y = np.random.rand()*(center_end_y - center_start_y) + center_start_y
# (x_min,y_min) - it's a left lowwer angle of a space 
x_min = center_start_x - (radius_end - radius_start)
y_min = center_start_y - (radius_end - radius_start)
# (x_max,y_max) - it's a right upper angle of a space
x_max = center_end_x + (radius_end - radius_start)
y_max = center_end_y + (radius_end - radius_start)
print("x_min = {}, y_min = {}, x_max = {}, y_max = {}".format(x_min, y_min, x_max, y_max))
print("radius_true = {:.2f}; center_true = ({:.2f},{:.2f})".format(radius,center_x, center_y))

x_min = -16, y_min = -16, x_max = 16, y_max = 16
radius_true = 2.29; center_true = (6.17,0.75)


### generate dots and extract inner and outer ones

In [4]:
random_dots = generate_dots(x_min, y_min, x_max, y_max, n_samples)
inner_points, outer_points = separate_dots(random_dots,center_x, center_y, radius, epsilon)
print("inner points.shape = ", inner_points.shape)
print("outer points.shape = ", outer_points.shape)
x_train = np.concatenate([inner_points, outer_points],axis=0)
# 0-lable for inner dots, 1-label  for outer dots
y_train = np.concatenate([np.zeros((inner_points.shape[0],)),np.ones((outer_points.shape[0]))])
print("x_train.shape : ", x_train.shape)
print("y_train.shape : ", y_train.shape)

inner points.shape =  (321143, 2)
outer points.shape =  (321143, 2)
x_train.shape :  (642286, 2)
y_train.shape :  (642286,)


### initialize parameters with random values

In [5]:
r_init = np.random.rand()*(radius_end - radius_start) + radius_start
c_x_init = np.random.rand()*(center_end_x - center_start_x) + center_start_x
c_y_init = np.random.rand()*(center_end_y - center_start_y) + center_start_y
print("radius = {}; center = ({},{})".format(r_init,c_x_init, c_y_init))

radius = 5.599162193706479; center = (-8.784233354046371,-3.7764434612123328)


In [6]:
a = np.array([1, 1, -2*c_x_init, -2*c_y_init, c_x_init**2 + c_y_init**2 - r_init**2])
# enchance the convergence of algorithm
learning_rate = 0.1
for epoch in range(epochs):
    x_train, y_train = shuffle_dataset(x_train,y_train)
    for dot, label in zip(x_train, y_train):
        x = np.array([dot[0]**2, dot[1]**2, dot[0], dot[1], 1])
        
        if a.dot(x.T) > 0 and label == 0:
            a[2] -= learning_rate*x[2]
            a[3] -= learning_rate*x[3]
            a[4] -= learning_rate*x[4]
        
        if a.dot(x.T) < 0 and label == 1:
            a[2] += learning_rate*x[2]
            a[3] += learning_rate*x[3]
            a[4] += learning_rate*x[4]

    if epoch%10 == 0 and epoch != 0 :
        learning_rate/=10
        print("learning_rate = ", learning_rate)
    
    c_x_predicted = (-1/2)*a[2]
    c_y_predicted = (-1/2)*a[3]
    radius_predicted = np.sqrt(c_x_predicted**2 +c_y_predicted**2 - a[4])
    print("EPOCH : {}, c_x : {} , c_y : {}, radius : {}".format(epoch,c_x_predicted,
                                                                c_y_predicted,radius_predicted))

EPOCH : 0, c_x : 6.295454145953613 , c_y : 0.5945440494688226, radius : 2.6481242357414994
EPOCH : 1, c_x : 6.150532270953601 , c_y : 0.7258525812299907, radius : 2.3199667288730854
EPOCH : 2, c_x : 6.170747114703601 , c_y : 0.7168766939314103, radius : 2.473533623236813
EPOCH : 3, c_x : 6.18568852095361 , c_y : 0.9030654871137815, radius : 2.569942837814004
EPOCH : 4, c_x : 6.188911177203617 , c_y : 0.8586760008009164, radius : 2.4220409722020584
EPOCH : 5, c_x : 6.220551802203612 , c_y : 0.7559075796278228, radius : 2.587082803119295
EPOCH : 6, c_x : 6.288520552203597 , c_y : 0.7350427114637645, radius : 2.68553811377184
EPOCH : 7, c_x : 6.2459424272035795 , c_y : 0.8722409689100574, radius : 2.701654594715077
EPOCH : 8, c_x : 6.172895552203559 , c_y : 0.6603245043428727, radius : 2.3805470772090023
EPOCH : 9, c_x : 6.226313520953536 , c_y : 0.6497782730730373, radius : 2.5525532908687283
learning_rate =  0.01
EPOCH : 10, c_x : 6.489301802203531 , c_y : 0.632843210664592, radius : 3.

In [7]:
print("c_x_true : {} , c_y_true : {}, radius_true : {}".format(center_x,center_y,radius))

c_x_true : 6.170838762729911 , c_y_true : 0.7469366378546312, radius_true : 2.2873869033968894
